In [1]:
from copy import deepcopy
from collections import deque 
import pulp

In [2]:
test = ['[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}',
        '[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}',
        '[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}']

In [3]:
def get_machines(data):
    machines = []
    for line in data:
        parts = line.strip().split()

        #part 0 is indicator
        indicator = []
        for i in parts[0][1:-1]:
            if i == '.':
                indicator.append(False)
            else:
                indicator.append(True)

        #part 1 to -1 are buttons
        buttons = []
        for button in parts[1:-1]:
            keys = button[1:-1].split(',')
            for i in range(0, len(keys)):
                keys[i] = int(keys[i])
            buttons.append(keys)

        #part -1 is joltage
        joltage = parts[-1][1:-1].split(',')
        for i in range(0, len(joltage)):
            joltage[i] = int(joltage[i])

        machines.append([indicator, buttons, joltage])

    return machines

def push_buttons(machine):
    target = machine[0]
    buttons = machine[1]

    state = []
    for t in target:
        state.append(False)

    queue = deque([])
    for button in buttons:
        queue.append([0, deepcopy(state), button])

    while queue:
        press = queue.popleft()
        for b in press[2]:
            press[1][b] = not press[1][b]

        count = 0
        for i in range(0, len(press[1])):
            if press[1][i] == target[i]:
                count += 1
        if count == len(state):
            return press[0]+1

        for button in buttons:
            queue.append([press[0]+1, deepcopy(press[1]), button])

def pull_levers(machine):
    #will probably work, bust slow AF
    target = machine[2]
    buttons = machine[1]

    state = []
    for t in target:
        state.append(0)

    queue = deque([])
    for button in buttons:
        queue.append([0, deepcopy(state), button])

    while queue:
        press = queue.popleft()
        for b in press[2]:
            press[1][b] += 1

        count = 0
        okay = True
        for i in range(0, len(press[1])):
            if press[1][i] == target[i]:
                count += 1
            elif press[1][i] > target[i]:
                okay = False
                break
        
        if count == len(state):
            return press[0]+1

        if okay:
            for button in buttons:
                queue.append([press[0]+1, deepcopy(press[1]), button])

def minimum_presses_joltage(machine):
    target = machine[2]
    buttons = machine[1]

    vectors = []
    for button in buttons:
        vector = []
        for i in range(0, len(target)):
            if i in button:
                vector.append(1)
            else:
                vector.append(0)
        vectors.append(vector)

    #It's a linear algebra problem, so use pulp to solve it for me
    
    prob = pulp.LpProblem("MinimizeCoefficientSum", pulp.LpMinimize)

    #variables
    x = []
    for i in range(0, len(buttons)):
         x.append(pulp.LpVariable(f"x_{i}", lowBound=0, cat="Integer"))

    # m coordinate equations
    for j in range(0, len(target)):
        prob += pulp.lpSum(x[i] * vectors[i][j] for i in range(0, len(buttons))) == target[j]

    # Objective function: minimize total number of vectors used
    prob += pulp.lpSum(x)

    # Solve
    prob.solve(pulp.PULP_CBC_CMD(msg=False))

    #minimum sum
    min_sum = pulp.value(prob.objective)

    return int(min_sum)
    

def run_part(data, part=1):
    machines = get_machines(data)

    if part == 1:
        presses = 0
        for machine in machines:
            presses += push_buttons(machine)
        return presses

    if part == 2:
        presses = 0
        for machine in machines:
            #presses += pull_levers(machine)
            presses += minimum_presses_joltage(machine)
        return presses
    return

In [4]:
value = run_part(test)
print('Part 1 test:', value, value==7)

Part 1 test: 7 True


In [5]:
with open('input_day10.txt', 'r') as f:
    data = f.readlines()
    f.close()

print('Part 1 result:', run_part(data))

Part 1 result: 434


In [6]:
value = run_part(test, 2)
print('Part 2 test:', value, value==33)

Part 2 test: 33 True


In [7]:
print('Part 2 result:', run_part(data, 2))

Part 2 result: 15132
